In [1]:
import numpy as np
import os
import pickle
import pandas as pd
from scipy.special import softmax

In [2]:
output_dir = "../../../epic-sounds-annotations/src/outputs"
mode = "val" #options: {val, test} NOTE: test cannot be analyzed as we don't have GT - it is for the EPIC-KITCHENS challenge.
if mode == "test":
    output_file = "scores/EPIC_Sounds_recognition_test_timestamps.pkl"
else:
     output_file = "scores/EPIC_Sounds_validation.pkl"
val_annotations_file = "../../../epic-sounds-annotations/EPIC_Sounds_validation.pkl"

In [3]:
with open(os.path.join(output_dir, mode, output_file), 'rb') as f:
    pred_annotations = pickle.load(f)

In [4]:
val_annotations = pd.read_pickle(val_annotations_file)

In [5]:
pred_annotations['interaction_output'].shape

(8035, 44)

In [5]:
val_annotations

,annotation_id,participant_id,video_id,start_timestamp,stop_timestamp,start_sample,stop_sample,description,class,class_id
0,P01_11_0,P01,P01_11,00:00:02.069,00:00:02.993,49656,71832,clang / clatter,ceramic / wood collision,34
1,P01_11_1,P01,P01_11,00:00:03.065,00:00:14.447,73560,346728,paper rustle,rustle,4
2,P01_11_2,P01,P01_11,00:00:16.706,00:00:32.406,400944,777744,paper rustle,rustle,4
3,P01_11_3,P01,P01_11,00:00:21.163,00:00:21.783,507912,522792,sniffle,human,24
4,P01_11_4,P01,P01_11,00:00:33.533,00:00:33.993,804792,815832,sniffle,human,24
...,...,...,...,...,...,...,...,...,...,...
8030,P32_10_73,P32,P32_10,00:05:47.605,00:05:48.385,8342520,8361240,open / close drawer,open / close,3
8031,P32_10_74,P32,P32_10,00:06:00.811,00:06:10.633,8659464,8895192,tap running,water,5
8032,P32_10_75,P32,P32_10,00:06:17.792,00:06:22.906,9067008,9189744,scrub / scrape / scour / wipe,water,5
8033,P32_10_76,P32,P32_10,00:06:34.734,00:06:35.534,9473616,9492816,open / close drawer,open / close,3


In [8]:
probabilities = softmax(pred_annotations['interaction_output'], axis=1)

# Get the indices of the top 5 predictions for each sample
top5_predicted_classes = np.argsort(-probabilities, axis=1)[:, :5]

# Predicted classes (top-1)
predicted_classes = top5_predicted_classes[:, 0]

# Create a DataFrame with predictions
df_predictions = pd.DataFrame({
    'annotation_id': pred_annotations['annotation_id'],
    'pred_class': predicted_classes,
    'top5_classes': list(top5_predicted_classes)
})

# Merge with ground truth annotations
merged_df = pd.merge(df_predictions, val_annotations[['annotation_id', 'class_id']], on='annotation_id')

# Compute top-1 accuracy
top1_accuracy = np.mean(merged_df['pred_class'] == merged_df['class_id'])

# Compute top-5 accuracy
top5_accuracy = merged_df.apply(lambda row: row['class_id'] in row['top5_classes'], axis=1).mean()

In [9]:
top5_accuracy

0.8446795270690728

In [10]:
top1_accuracy

0.5373988799004356